## CNN on CIFAR dataset with keras

##### Fix random seed for reproducibility

In [1]:
import numpy as np

np.random.seed(123) 

##### Set Current working directory

In [2]:
import os

# Change directory
PATH = os.getcwd() 
print (PATH)
os.chdir(PATH)

/home/yogesh/lab_activities/Deep_Learning_42/CNN


##### Keras model module

    Import the Sequential model type from Keras. This is simply a linear stack of neural network layers

In [3]:
from keras.models import Sequential

/home/yogesh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Use the retry module or similar alternatives.


##### Keras core layers
    
    Import the "core" layers from Keras. These are the layers that are used in almost any neural network:

In [4]:
from keras.layers import core

##### Keras CNN layers

     Import the CNN layers from Keras. These are the convolutional layers that will help us efficiently train on image data:

In [5]:
from keras.layers import convolutional, pooling

##### Utilities

    import some utilities. This will help us transform our data later:

In [6]:
from keras.utils import np_utils

#### Load the CIFAR image data from `keras.datasets`


* CIFAR-10 is an established computer-vision dataset used for object recognition. It is a subset of the 80 million tiny images dataset and consists of 60,000 32x32 color images containing one of 10 object classes, with 6000 images per class.

In [32]:
from keras.datasets import cifar10
 
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170434560/170498071 [============================>.] - ETA: 0s

Returns:

    2 tuples:
            x_train, x_test: uint8 array of image data with shape (num_samples, 32, 32, 3).
            y_train, y_test: uint8 array of digit labels (integers in range 0-9) with shape (num_samples, 1).

Arguments:

    path: if you do not have the index file locally (at '~/.keras/datasets/' + path), it will be downloaded to this location.


Look at the shape of the dataset:

In [35]:
print (X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


##### Keras backends

Keras is a model-level library, providing high-level building blocks for developing deep learning models. It does not handle itself low-level operations such as tensor products, convolutions and so on. Instead, it relies on a specialized, well-optimized tensor manipulation library to do so, serving as the "backend engine" of Keras. Rather than picking one single tensor library and making the implementation of Keras tied to that library, Keras handles the problem in a modular way, and several different backend engines can be plugged seamlessly into Keras.

At this time, Keras has three backend implementations available: the TensorFlow backend, the Theano backend, and the CNTK backend.

    TensorFlow is an open-source symbolic tensor manipulation framework developed by Google, Inc.
    Theano is an open-source symbolic tensor manipulation framework developed by LISA/MILA Lab at Université de Montréal.
    CNTK is an open-source, commercial-grade toolkit for deep learning developed by Microsoft.

In the future, some more backend options may be added.

##### Switching from one backend to another

If you have run Keras at least once, you will find the Keras configuration file at:

$HOME/.keras/keras.json

If it isn't there, you can create it.

#### keras.json details

{

    "image_data_format": "channels_last",

    "epsilon": 1e-07,
        
    "floatx": "float32",
    
    "backend": "tensorflow"

}

You can change these settings by editing $HOME/.keras/keras.json.

    image_data_format: string, either "channels_last" or "channels_first". It specifies which data format convention Keras will follow. (keras.backend.image_data_format() returns it.)
        For 2D data (e.g. image), "channels_last" assumes (rows, cols, channels) while 
                                  "channels_first" assumes (channels, rows, cols).
        For 3D data, "channels_last" assumes (conv_dim1, conv_dim2, conv_dim3, channels) while 
                     "channels_first" assumes (channels, conv_dim1, conv_dim2, conv_dim3).
    epsilon: float, a numeric fuzzing constant used to avoid dividing by zero in some operations.
    
    floatx: string, "float16", "float32", or "float64". Default float precision.
    
    backend: string, "tensorflow", "theano", or "cntk".


In [36]:
from keras import backend

print (backend.backend())
print (backend.image_data_format())

tensorflow
channels_last


#### Preprocess input data for Keras

In [37]:
print(X_train.shape[0])

50000


Reshape input data

    When using the tensorflow backend, you must explicitly declare a dimension for the depth of the input image. 
    E.g. a full-color image with all 3 RGB channels will have a depth of 3 such as (n, width, height, channel).

 print X_train's dimensions

In [39]:
print (X_train.shape)

(50000, 32, 32, 3)


Convert our data type to float32 and normalize our data values to the range [0, 1]

Note: Max value X_train/X_test can take is 255, so it is divided by 255

In [40]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

#### Preprocess class labels for Keras

Look at the shape of our class label data

In [41]:
print (y_train.shape)

(50000, 1)


We should have 10 different classes, but it looks like we only have a 1-dimensional array.

In [44]:
print (y_train[:10])

[[6]
 [9]
 [9]
 [4]
 [1]
 [1]
 [2]
 [7]
 [8]
 [3]]


##### Convert 1-dimensional class arrays to 10-dimensional class matrices

In [45]:
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

Take another look:

In [46]:
print(Y_train.shape)

(50000, 10)


In [47]:
print(Y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


#### Define model architecture.

Declaring a sequential model

In [56]:
model = Sequential()

Declare the input layer

CNN input layer

In [57]:
model.add(convolutional.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))

###### 2D convolution layer (e.g. spatial convolution over images).

This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs. If use_bias is True, a bias vector is created and added to the outputs. Finally, if activation is not None, it is applied to the outputs as well.

When using this layer as the first layer in a model, provide the keyword argument input_shape (tuple of integers, does not include the sample axis), e.g. input_shape=(28, 28, 1) for 28x28 gray pictures in data_format="channels_last".

First parameters correspond to the number of convolution filters 

Next 2 parameters correspond to kernal size

###### Add more layers to our model

In [58]:
model.add(convolutional.Conv2D(32, (3, 3), activation='relu'))
model.add(pooling.MaxPooling2D(pool_size=(2, 2)))
model.add(core.Dropout(0.25))

Dropout

    This is a method for regularizing our model in order to prevent overfitting. 

MaxPooling2D 

    Is a way to reduce the number of parameters in our model by sliding a 2x2 pooling filter across the previous layer and taking the max of the 4 values in the 2x2 filter.

So far, for model parameters, we've added two Convolution layers. To complete our model architecture, let's add a fully connected layer and then the output layer

In [59]:
model.add(core.Flatten())
model.add(core.Dense(128, activation='relu'))
model.add(core.Dropout(0.5))
model.add(core.Dense(10, activation='softmax'))

For Dense layers, the first parameter is the output size of the layer. Keras automatically handles the connections between layers.

Note that the final layer has an output size of 10, corresponding to the 10 classes of digits.

Also note that the weights from the Convolution layers must be flattened (made 1-dimensional) before passing them to the fully connected Dense layer.

#### Compile model

Compile the model by providing the loss function and the optimizer 

In [60]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [61]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               802944    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
__________

#### Fit model on training data.

In [62]:
model.fit(X_train, Y_train, batch_size=32, epochs=2, verbose=2)

Epoch 1/2
80s - loss: 1.5430 - acc: 0.4435
Epoch 2/2
89s - loss: 1.2157 - acc: 0.5706


#### Evaluate the model on test data.

In [63]:
score = model.evaluate(X_test, Y_test, verbose=0)
print(score)

[1.0549163019180299, 0.6317]


#### Make predictions on test data 

In [64]:
# calculate predictions
predictions = model.predict(X_test)

In [65]:
print (predictions)

[[4.96057607e-03 6.18038757e-04 3.03653758e-02 ... 1.13424417e-02
  3.42388102e-03 8.98917112e-03]
 [1.20685101e-01 7.80714571e-01 6.62122038e-05 ... 6.80314116e-08
  9.06425416e-02 7.83650205e-03]
 [3.31039876e-01 3.34322810e-01 2.02969238e-02 ... 1.00379367e-03
  1.91191450e-01 1.03523836e-01]
 ...
 [2.11372133e-03 3.87668442e-05 9.30316225e-02 ... 8.86757299e-02
  1.64373399e-04 4.57310089e-04]
 [7.90476874e-02 6.07554495e-01 2.70472392e-02 ... 8.24164879e-03
  9.61851981e-03 3.95110138e-02]
 [5.91633070e-05 3.15426296e-05 2.07308936e-03 ... 9.37291682e-01
  2.33186961e-06 3.00148604e-05]]


In [66]:
y_pred_class = model.predict_classes(X_test)

 9984/10000 [============================>.] - ETA: 0s

In [67]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.6317

In [68]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[769,  24,  87,  15,  12,   6,  14,  12,  23,  38],
       [ 39, 783,  10,  13,   5,   3,  14,  12,   6, 115],
       [ 89,   5, 474,  64, 109, 127,  78,  38,   3,  13],
       [ 30,  11,  85, 422,  71, 224,  97,  39,   4,  17],
       [ 50,   4, 113,  75, 533,  43,  73, 100,   6,   3],
       [ 17,   3,  76, 145,  42, 610,  32,  67,   6,   2],
       [  7,   6,  71,  59,  46,  35, 762,  11,   1,   2],
       [ 17,   1,  36,  40,  56, 102,   7, 731,   1,   9],
       [277,  83,  20,  31,   8,   8,  10,   5, 515,  43],
       [ 62, 108,  16,  14,  14,   8,  13,  36,  11, 718]])

Reference: 

    https://keras.io/
    https://elitedatascience.com
    http://scikit-learn.org/stable/modules/classes.html